# Variance Reduction for European Options
In this notebook, we reduce variance of Monte Carlo pricing of European option via anthithetic sampling and control variates. For simplicity, we will only deal with call options. The implementation for put options is very similar.

We assume that the stock prices follow Geometric Brownian Motion, i.e. the price $S_t$ at time $t$ of a stock can be expressed as
$$
S_t = S_0 \exp \left\{ \left( r - \frac{1}{2} \sigma^2 \right) t + \sigma W_t \right\}
$$
where $S_0$ is the initial stock price (at time 0), $r$ is the risk-free interest rate, $\sigma$ is the volatility of the stock, and $W_t$ is the standard Brownian motion.

In [1]:
import numpy as np
import pandas as pd
from scipy.stats import norm
import time

print('numpy version: ',np.__version__)
print('pandas version: ',pd.__version__)

numpy version:  1.17.4
pandas version:  0.25.3


### Black-Scholes Formula for European Call Options

We denote the maturity by $T$ and the strike price by $K$. Then the Black-Scholes price of the option is given by
$$
\mathbb{E}[(K - S_T)^+] = S_0 \Phi(\sigma \sqrt T - \theta) - K e^{-rT} \Phi(-\theta) \quad \text{where} \quad 
\theta = \frac{1}{\sigma \sqrt T} \log \frac{K}{S_0} + \left( \frac{1}{2} \sigma - \frac{r}{\sigma} \right) \sqrt T
$$
and $\Phi$ is the $CDF$ of the standard normal.


In [2]:
def BS_call(S_0, r, sigma, K, T):
    '''
    Black-Scholes price for European call option:
    S_0 - price of the underlying stock at time 0
    r - annual risk-free interest rate in decimal
    sigma - volatility of the underlying stock
    K - strike price
    T - time to maturity in years
    The result is rounded to 5 decimal places
    '''
    theta = (1 / (sigma * np.sqrt(T))) * np.log(K / S_0) + (0.5 * sigma - r / sigma) * np.sqrt(T)
    bs_call_price = S_0 * norm.cdf(sigma * np.sqrt(T) - theta) - K * np.exp(-r * T) * norm.cdf(-theta)
    return round(bs_call_price, 5)

### Calculation of European Call Options Using Monte Carlo

#### Plain Monte Carlo Estimation

In [3]:
def MC_call(S_0, r, sigma, K, T, n):
    '''
    Price for European call option using Monte Carlo simulation:
    S_0 - price of the underlying stock at time 0
    r - annual risk-free interest rate in decimal
    sigma - volatility of the underlying stock
    K - strike price
    T - time to maturity in years
    n - number of iterations
    Returns both the price and standard error
    The results are  rounded to 5 decimal places
    '''
    Z = np.random.standard_normal(size=n)
    Y = S_0 * np.exp((r - 0.5 * sigma ** 2) * T + sigma * np.sqrt(T) * Z)
    X =  np.maximum(Y - K, np.zeros(n))
    mc_call_price = np.exp(-r * T) * np.mean(X)
    mc_call_se = np.exp(-r * T) * np.std(X) / np.sqrt(n)
    return round(mc_call_price, 5), round(mc_call_se, 5)

#### Monte Carlo Estimation with Antithetic Sampling

In [4]:
def MC_call_as(S_0, r, sigma, K, T, n):
    '''
    Price for European call option using Monte Carlo simulation with antithetic sampling:
    S_0 - price of the underlying stock at time 0
    r - annual risk-free interest rate in decimal
    sigma - volatility of the underlying stock
    K - strike price
    T - time to maturity in years
    n - total number of samples (n/2 generated) and (n/2 by antithetic sampling)
    Returns both the price and standard error
    The results are  rounded to 5 decimal places
    '''
    Z1 = np.random.standard_normal(size=int(n/2))
    Z2 = -Z1
    Z = np.concatenate((Z1, Z2))
    Y = S_0 * np.exp((r - 0.5 * sigma ** 2) * T + sigma * np.sqrt(T) * Z)
    X = np.maximum(Y - K, np.zeros(n))
    mc_call_price = np.exp(-r * T) * np.mean(X)
    mc_call_se = np.exp(-r * T) * np.std(X) / np.sqrt(n)
    return round(mc_call_price, 5), round(mc_call_se, 5)

#### Monte Carlo Estimation with Control Variates
We will estimate the price of an European call option using Monte Carlo with the discounted stock price $e^{-rT} S_T$ serving as the control variate.

In [5]:
def MC_call_cv(S_0, r, sigma, K, T, n):
    '''
    Price for European call option using Monte Carlo simulation with
    the discounted stock price as control variate:
    S_0 - price of the underlying stock at time 0
    r - annual risk-free interest rate in decimal
    sigma - volatility of the underlying stock
    K - strike price
    T - time to maturity in years
    n - number of samples
    Returns both the price and standard error
    The results are  rounded to 5 decimal places
    '''
    Z = np.random.standard_normal(size=n)
    w = np.random.standard_normal(size=1000) # will be used to estimate b_star
    #estimation of b_star
    s = S_0 * np.exp((r - 0.5 * sigma ** 2) * T + sigma * np.sqrt(T) * w)
    x = np.exp(-r * T) * np.maximum(s - K, np.zeros(1000))
    y = np.exp(-r * T) * s - S_0
    b_star = np.cov(x, y)[0, 1] / np.var(y)
    # back to MC
    S = S_0 * np.exp((r - 0.5 * sigma ** 2) * T + sigma * np.sqrt(T) * Z)
    X = np.exp(-r * T) * np.maximum(S - K, np.zeros(n))
    Y = np.exp(-r * T) * S - S_0
    H = X - b_star * Y
    mc_call_price = np.mean(H)
    mc_call_se = np.std(H) / np.sqrt(n)
    return round(mc_call_price, 5), round(mc_call_se, 5)

#### European Call Option with strike price $K=40$,  $S_0 = 50$, $r=0.05$, $\sigma=0.2$, and $T=1$.

In [6]:
np.random.seed(2020)
sample_size = [1e4, 1e5, 1e6, 1e7, 1e8]
mc_estimates = []
mc_se = []
bs_price = np.repeat(BS_call(50, 0.05, 0.2, 40, 1), 5)
mc_time = []


for n in sample_size:
    time0 = time.time()
    mc_call = MC_call(50, 0.05, 0.2, 40, 1, int(n))
    time1 = time.time()
    mc_estimates.append(mc_call[0])
    mc_se.append(mc_call[1])
    mc_time.append(time1 - time0)
    
df = pd.DataFrame({'Sample Size': sample_size, 'Black-Scholes Price': bs_price, 
                   'Plain MC Estimate': mc_estimates, 'Plain MC se': mc_se, 'Plain MC time in s': mc_time})
df

,Sample Size,Black-Scholes Price,Plain MC Estimate,Plain MC se,Plain MC time in s
0,10000.0,12.29442,12.36667,0.09503,0.000998
1,100000.0,12.29442,12.30530,0.03027,0.006980
2,1000000.0,12.29442,12.29188,0.00959,0.067818
3,10000000.0,12.29442,12.29520,0.00303,0.598191
4,100000000.0,12.29442,12.29527,0.00096,6.056544


In [7]:
np.random.seed(2020)
sample_size = [1e4, 1e5, 1e6, 1e7, 1e8]
mc_estimates = []
mc_se = []
bs_price = np.repeat(BS_call(50, 0.05, 0.2, 40, 1), 5)
mc_time = []


for n in sample_size:
    time0 = time.time()
    mc_call = MC_call_as(50, 0.05, 0.2, 40, 1, int(n))
    time1 = time.time()
    mc_estimates.append(mc_call[0])
    mc_se.append(mc_call[1])
    mc_time.append(time1 - time0)
    
df = pd.DataFrame({'Sample Size': sample_size, 'Black-Scholes Price': bs_price, 
                   'MC Estimate w/ AS': mc_estimates, 'MC w/ AS se': mc_se, 'MC w/ AS time in s': mc_time})
df

,Sample Size,Black-Scholes Price,MC Estimate w/ AS,MC w/ AS se,MC w/ AS time in s
0,10000.0,12.29442,12.27028,0.09489,0.000000
1,100000.0,12.29442,12.27955,0.03018,0.000000
2,1000000.0,12.29442,12.29580,0.00959,0.081224
3,10000000.0,12.29442,12.29288,0.00303,0.549410
4,100000000.0,12.29442,12.29458,0.00096,8.258686


In [8]:
np.random.seed(2020)
sample_size = [1e4, 1e5, 1e6, 1e7, 1e8]
mc_estimates = []
mc_se = []
bs_price = np.repeat(BS_call(50, 0.05, 0.2, 40, 1), 5)
mc_time = []


for n in sample_size:
    time0 = time.time()
    mc_call = MC_call_cv(50, 0.05, 0.2, 40, 1, int(n))
    time1 = time.time()
    mc_estimates.append(mc_call[0])
    mc_se.append(mc_call[1])
    mc_time.append(time1 - time0)
    
df = pd.DataFrame({'Sample Size': sample_size, 'Black-Scholes Price': bs_price, 
                   'MC Estimate w/ CV': mc_estimates, 'MC w/ CV se': mc_se, 'MC w/ CV time in s': mc_time})
df

,Sample Size,Black-Scholes Price,MC Estimate w/ CV,MC w/ CV se,MC w/ CV time in s
0,10000.0,12.29442,12.27843,0.01145,0.128433
1,100000.0,12.29442,12.29112,0.00376,0.015621
2,1000000.0,12.29442,12.29521,0.00120,0.124586
3,10000000.0,12.29442,12.29428,0.00038,1.013776
4,100000000.0,12.29442,12.29434,0.00012,7.596653


#### European Call Option with strike price $K=60$,  $S_0 = 50$, $r=0.05$, $\sigma=0.2$, and $T=1$.

In [9]:
np.random.seed(2020)
sample_size = [1e4, 1e5, 1e6, 1e7, 1e8]
mc_estimates = []
mc_se = []
bs_price = np.repeat(BS_call(50, 0.05, 0.2, 60, 1), 5)
mc_time = []


for n in sample_size:
    time0 = time.time()
    mc_call = MC_call(50, 0.05, 0.2, 60, 1, int(n))
    time1 = time.time()
    mc_estimates.append(mc_call[0])
    mc_se.append(mc_call[1])
    mc_time.append(time1 - time0)
    
df = pd.DataFrame({'Sample Size': sample_size, 'Black-Scholes Price': bs_price, 
                   'Plain MC Estimate': mc_estimates, 'Plain MC se': mc_se, 'Plain MC time in s': mc_time})
df

,Sample Size,Black-Scholes Price,Plain MC Estimate,Plain MC se,Plain MC time in s
0,10000.0,1.62374,1.62519,0.04230,0.000997
1,100000.0,1.62374,1.62517,0.01372,0.009974
2,1000000.0,1.62374,1.62747,0.00435,0.077932
3,10000000.0,1.62374,1.62311,0.00137,0.669177
4,100000000.0,1.62374,1.62463,0.00043,6.364646


In [10]:
np.random.seed(2020)
sample_size = [1e4, 1e5, 1e6, 1e7, 1e8]
mc_estimates = []
mc_se = []
bs_price = np.repeat(BS_call(50, 0.05, 0.2, 60, 1), 5)
mc_time = []


for n in sample_size:
    time0 = time.time()
    mc_call = MC_call_as(50, 0.05, 0.2, 60, 1, int(n))
    time1 = time.time()
    mc_estimates.append(mc_call[0])
    mc_se.append(mc_call[1])
    mc_time.append(time1 - time0)
    
df = pd.DataFrame({'Sample Size': sample_size, 'Black-Scholes Price': bs_price, 
                   'MC Estimate w/ AS': mc_estimates, 'MC w/ AS se': mc_se, 'MC w/ AS time in s': mc_time})
df

,Sample Size,Black-Scholes Price,MC Estimate w/ AS,MC w/ AS se,MC w/ AS time in s
0,10000.0,1.62374,1.59765,0.04239,0.000998
1,100000.0,1.62374,1.60648,0.01359,0.006980
2,1000000.0,1.62374,1.62555,0.00434,0.059200
3,10000000.0,1.62374,1.62176,0.00137,0.531124
4,100000000.0,1.62374,1.62387,0.00043,5.836071


In [11]:
np.random.seed(2020)
sample_size = [1e4, 1e5, 1e6, 1e7, 1e8]
mc_estimates = []
mc_se = []
bs_price = np.repeat(BS_call(50, 0.05, 0.2, 60, 1), 5)
mc_time = []


for n in sample_size:
    time0 = time.time()
    mc_call = MC_call_cv(50, 0.05, 0.2, 60, 1, int(n))
    time1 = time.time()
    mc_estimates.append(mc_call[0])
    mc_se.append(mc_call[1])
    mc_time.append(time1 - time0)
    
df = pd.DataFrame({'Sample Size': sample_size, 'Black-Scholes Price': bs_price, 
                   'MC Estimate w/ CV': mc_estimates, 'MC w/ CV se': mc_se, 'MC w/ CV time in s': mc_time})
df

,Sample Size,Black-Scholes Price,MC Estimate w/ CV,MC w/ CV se,MC w/ CV time in s
0,10000.0,1.62374,1.59633,0.02780,0.001995
1,100000.0,1.62374,1.62177,0.00905,0.008977
2,1000000.0,1.62374,1.62843,0.00287,0.098098
3,10000000.0,1.62374,1.62278,0.00091,0.901926
4,100000000.0,1.62374,1.62431,0.00029,8.482253


#### Observations
* Monte Carlo estimation with control variates seems to outperform the other two when the reduction in standard error is taken into account.
* We see an 8-fold reduction in variance when we compare plain MC and MC with CV when $K=40$, but only 1.5 times reduction when $K=60$. The reason is that as $K$ becomes larger, the correlation between $(S_T - K)^+$ and $S_T$ becomes weaker. 